In [7]:
!pip install category-encoders

     |████████████████████████████████| 81kB 3.5MB/s 


In [23]:
import pandas as pd
import plotly 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import *
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

from sklearn.preprocessing import label_binarize

#dispplay options
pd.set_option("max_columns",500)

In [10]:
def change_month_to_month_difference(df):
    if 'customer_id' not in df.columns:
        le = LabelEncoder()
        df['customer_id'] = le.fit_transform(df['customer_id_mskd'])
    
    if 'timestamp' not in df.columns:
        df['timestamp'] = pd.to_datetime(df['month'])
    
    output = {}
    for cust_id in df['customer_id'].unique():
        diff = df[df['customer_id'] == cust_id]['timestamp'].max() - df[df['customer_id'] == cust_id]['timestamp'].min()
        diff_int = int(diff/np.timedelta64(1,'M'))
        
        output[cust_id] = diff_int
    output = pd.DataFrame.from_dict(output, orient = 'index')
    output.columns = ['month_difference']
    #df = pd.merge(df, output, how = 'inner', left_on = 'customer_id', right_on = output.index)
    
    return output

In [11]:
def data_prep(cust_df,tran_df,run_custDf = True, run_tranDf = True):
  
  if run_custDf:
    #preprocess customer data columns
    custDf = cust_df.copy()
    #make encode customer_id
    le_cust = LabelEncoder() #<--- to be used in tranDf for customer_id_maskd
    custDf['customer_Id'] = le_cust.fit_transform(custDf['customer_id_mskd'])

    #drop customer_id_mskd
    custDf.drop('customer_id_mskd',axis=1,inplace= True)

    #replace NULL in count features (PCD and SRV) with 0
    featCount = [col for col in custDf.columns if ('PCD' in col) or ('SRV' in col)]
    custDf[featCount] = custDf[featCount].fillna(0)

    #drop export_ts
    custDf.drop('export_ts',axis = 1, inplace = True)

    #drop industry_code_aml if all none
    if custDf.industry_code_aml.isnull().sum() == len(custDf):
      custDf.drop('industry_code_aml', axis = 1, inplace = True)

    # ------add other preprocessing for custDf here-----------

  if run_tranDf:

    #----------------------preprocess tranDf------------------
    tranDf = tran_df.copy()
    #encode customer ids using same encoder as in custDf
    tranDf['customer_id'] = le_cust.transform(tranDf['customer_id_mskd'])

    #drop customer_id_mskd
    tranDf.drop('customer_id_mskd',inplace = True, axis = 1)

    #process the month column
    months_diff =change_month_to_month_difference(tranDf) #returns customer_id as index and month's diff as columns
    

    #replace NULL with 0 in 'in_amt', 'in_cnt', 'out_amt', 'out_cnt'
    featTran = ['in_amt', 'in_cnt', 'out_amt', 'out_cnt']
    tranDf[featTran] = tranDf[featTran].fillna(0)

    #groupby custumer_id, sum for 'in_amt', 'in_cnt', 'out_amt', 'out_cnt'

    gpby = tranDf.groupby(by = 'customer_id')
    tranSum = gpby[featTran].sum()

    tranSum = pd.merge(tranSum, months_diff, left_on = tranSum.index, right_on = months_diff.index, how = 'inner')
    tranSum.rename(columns={'key_0':'customer_Id'},inplace = True)

    #to return tranSum
    return [custDf,tranSum]

In [12]:
custDf = pd.read_parquet('bigdata2021data/cust_train.parquet')
tranDf = pd.read_parquet("bigdata2021data/transaction_train.parquet")

In [13]:
[cust_df, tran_df] = data_prep(custDf,tranDf,run_custDf = True, run_tranDf = True)

In [14]:
cust_df.head(5)

,jurisdiction_code,client_type_aml,occupation_code_aml,country_of_domicile_aml,occupation_status_code_aml,customer_status_aml,primary_ownership_flag,relationship_type,PCD_CDA,PCD_CMS,PCD_CRC,PCD_LLC,PCD_MOR,PCD_MUF,PCD_SAV,PCD_SDB,PCD_TED,SRV_FLG,SRV_FSL,SRV_ILC,SRV_LOC,SRV_NLG,SRV_NSL,SRV_TRF,PRD_INFO_AVAIL,rating,customer_Id
0,CA03,Individual,108,CA,None,Active,N,Joint,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,3.0,3946
1,CA03,Individual,342,CA,None,Active,Y,Borrower,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,3.0,3752
2,CA03,Individual,E902,CA,E902,Active,Y,Borrower,0.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,3.0,2978
3,CA03,Individual,94,CA,None,Active,Y,Primary Joint,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,3.0,657
4,CA03,Individual,256,CA,None,Active,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,3.0,3339


In [15]:
cust_df.columns

Index(['jurisdiction_code', 'client_type_aml', 'occupation_code_aml',
       'country_of_domicile_aml', 'occupation_status_code_aml',
       'customer_status_aml', 'primary_ownership_flag', 'relationship_type',
       'PCD_CDA', 'PCD_CMS', 'PCD_CRC', 'PCD_LLC', 'PCD_MOR', 'PCD_MUF',
       'PCD_SAV', 'PCD_SDB', 'PCD_TED', 'SRV_FLG', 'SRV_FSL', 'SRV_ILC',
       'SRV_LOC', 'SRV_NLG', 'SRV_NSL', 'SRV_TRF', 'PRD_INFO_AVAIL', 'rating',
       'customer_Id'],
      dtype='object')

In [22]:
tran_df.columns

Index(['customer_Id', 'in_amt', 'in_cnt', 'out_amt', 'out_cnt',
       'month_difference'],
      dtype='object')

In [21]:
df = pd.merge(cust_df,tran_df, left_on = 'customer_Id', right_on = 'customer_Id', how = 'left')

In [48]:
df.columns

Index(['jurisdiction_code', 'client_type_aml', 'occupation_code_aml',
       'country_of_domicile_aml', 'occupation_status_code_aml',
       'customer_status_aml', 'primary_ownership_flag', 'relationship_type',
       'PCD_CDA', 'PCD_CMS', 'PCD_CRC', 'PCD_LLC', 'PCD_MOR', 'PCD_MUF',
       'PCD_SAV', 'PCD_SDB', 'PCD_TED', 'SRV_FLG', 'SRV_FSL', 'SRV_ILC',
       'SRV_LOC', 'SRV_NLG', 'SRV_NSL', 'SRV_TRF', 'PRD_INFO_AVAIL', 'rating',
       'customer_Id', 'in_amt', 'in_cnt', 'out_amt', 'out_cnt',
       'month_difference'],
      dtype='object')

In [27]:
col_name = 'PCD_CDA'
targetVar = 'rating'
fig = make_subplots(rows = 1, cols = 3, subplot_titles=['Low Risk', "Medium Risk","High Risk"])
row = 1
col = 1
for rating in range(1,4):
  sub_df = df[df[targetVar]==rating]
  x = sub_df[col_name]
  y = sub_df[targetVar]

  fig.add_trace(go.Scatter(x = x, y = y), row = row, col = col)
  #row += 1
  col += 1

fig.update_layout(showlegend = False)
fig.show()



In [56]:
sub_df = df.filter(regex='^PCD',axis=1)
sub_df['rating'] = df['rating']

fig = make_subplots(rows = 3, cols = 3, subplot_titles=["{} vs Rating".format(i) for i in sub_df.drop('rating',axis = 1).columns])
i = 0
col_names = sub_df.drop('rating',axis = 1).columns
for row in range(1,4):
  for col in range(1,4):
    col_name = col_names[i]
    x = sub_df[col_name]
    y = sub_df[targetVar]
    fig.add_trace(go.Scatter(x=x, y = y, mode = 'markers'),row = row, col = col)
    fig.update_xaxes(title = col_name, row = row, col = col)

    i+=1
fig.update_yaxes(title = "Rating")
fig.update_layout(height = 1000, width = 1000, showlegend = False)
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
sub_df = df.filter(regex='^SRV',axis=1)
sub_df['rating'] = df['rating']
print(sub_df.shape)

fig = make_subplots(rows = 2, cols = 3, subplot_titles=["{} vs Rating".format(i) for i in sub_df.drop('rating',axis = 1).columns])
i = 0
col_names = sub_df.drop('rating',axis = 1).columns
for row in range(1,3):
  for col in range(1,4):
    col_name = col_names[i]
    x = sub_df[col_name]
    y = sub_df[targetVar]
    fig.add_trace(go.Scatter(x=x, y = y, mode = 'markers'),row = row, col = col)
    fig.update_xaxes(title = col_name, row = row, col = col)
    fig.update_yaxes(title = "Rating", row = row, col = col)
    i+=1
fig.update_layout(height = 1000, showlegend = False)
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(4469, 8)


In [49]:
sub_df = df.filter(regex='^SRV_TRF',axis=1)
sub_df['rating'] = df['rating']
print(sub_df.shape)

fig = make_subplots(rows = 1, cols = 1, subplot_titles=["{} vs Rating".format(i) for i in sub_df.drop('rating',axis = 1).columns])
i = 0
col_names = sub_df.drop('rating',axis = 1).columns
for row in range(1,2):
  for col in range(1,2):
    col_name = col_names[i]
    x = sub_df[col_name]
    y = sub_df[targetVar]
    fig.add_trace(go.Scatter(x=x, y = y, mode = 'markers'),row = row, col = col)
    fig.update_xaxes(title = col_name, row = row, col = col)
    fig.update_yaxes(title = "Rating", row = row, col = col)
    i+=1
fig.update_layout(height = 1000, showlegend = False)
fig.show()

(4469, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
df.columns

Index(['jurisdiction_code', 'client_type_aml', 'occupation_code_aml',
       'country_of_domicile_aml', 'occupation_status_code_aml',
       'customer_status_aml', 'primary_ownership_flag', 'relationship_type',
       'PCD_CDA', 'PCD_CMS', 'PCD_CRC', 'PCD_LLC', 'PCD_MOR', 'PCD_MUF',
       'PCD_SAV', 'PCD_SDB', 'PCD_TED', 'SRV_FLG', 'SRV_FSL', 'SRV_ILC',
       'SRV_LOC', 'SRV_NLG', 'SRV_NSL', 'SRV_TRF', 'PRD_INFO_AVAIL', 'rating',
       'customer_Id', 'in_amt', 'in_cnt', 'out_amt', 'out_cnt',
       'month_difference'],
      dtype='object')

In [62]:
fig = make_subplots(rows = 1, cols = 3, subplot_titles=['Low Risk', "Medium Risk", "High Risk"])
col = 1
for rating in range(1,4):
  sub_df = df[df[targetVar]==rating]
  fig.add_trace( go.Histogram(x = sub_df['in_amt'], nbinsx = 500), row = 1, col = col)
  col +=1
fig.show()


In [67]:
#fig = make_subplots(rows = 1, cols = 3, subplot_titles=['Low Risk', "Medium Risk", "High Risk"])
#col = 1
fig = go.Figure()
for rating in range(1,4):
  sub_df = df[df[targetVar]==rating]
  fig.add_trace( go.Histogram(x = sub_df['in_amt'], nbinsx = 20, name = "Rating {}".format(rating)))
  #col +=1
fig.update_layout(xaxis = dict(title = "Sum of In Amount"))
fig.show()

In [69]:
#fig = make_subplots(rows = 1, cols = 3, subplot_titles=['Low Risk', "Medium Risk", "High Risk"])
#col = 1
fig = go.Figure()
for rating in range(1,4):
  sub_df = df[df[targetVar]==rating]
  fig.add_trace( go.Histogram(x = sub_df['out_amt'], nbinsx = 20, name = "Rating {}".format(rating)))
  #col +=1
fig.update_layout(xaxis = dict(title = "Sum of Out Amount"))
fig.show()

In [71]:
fig = go.Figure()
for rating in range(1,4):
  sub_df = df[df[targetVar]==rating]
  fig.add_trace( go.Histogram(x = sub_df['in_cnt'], name = "Rating {}".format(rating)))
  #col +=1
fig.update_layout(xaxis = dict(title = "Total IN transactions"))
fig.show()

In [72]:
fig = go.Figure()
for rating in range(1,4):
  sub_df = df[df[targetVar]==rating]
  fig.add_trace( go.Histogram(x = sub_df['out_cnt'], name = "Rating {}".format(rating)))
  #col +=1
fig.update_layout(xaxis = dict(title = "Total OUT transactions"))
fig.show()